In [1]:
import os
import pickle 
import pandas as pd
import jieba
from zhon.hanzi import punctuation
import re
import googletrans
from pprint import pprint
import matplotlib.pyplot as plt 
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

# 1. 載入Corpus

In [68]:
path='C:/Users/USER/Desktop/Prof_Hsieh_Project/about_emotion/Chinese_derivative/corpus'
os.chdir(path)

# Facebook posts
fb_data = pd.read_pickle('facebook_to_1208.pkl')
fb_data['Publication_Date'] = fb_data['Publication_Time'].apply(lambda x: str(x.date()))
fb_data = fb_data.drop(columns=['Publisher', 'Post_url', 'Likes',
       'Reactions', 'Comments', 'Shares', 'Cover_photo','Publication_Time'])
fb_data['Publication_Date']= pd.to_datetime(fb_data['Publication_Date'])

# News
news_data = pd.read_pickle('news_to_1208.pkl')
news_data.rename(columns = {'pub_date':'Publication_Time', 'content':'Post_Content'}, inplace = True)
news_data['Publication_Date'] = news_data['Publication_Time'].apply(lambda x: str(x.date()))
news_data = news_data.drop(columns=['title','search_key', 'link', 'search_time', 'description','source','Publication_Time'])
news_data['Publication_Date']= pd.to_datetime(news_data['Publication_Date'])

# 2. Preprocess

In [65]:
def preprocess(text):
    """
    remove punctuations, numbers, english letters, line break and blank.
    """
    for i in punctuation:
        text = text.replace(i, '')
    text = re.sub('[\d]','',text) # [0-9]
    
    text = re.sub(r'[.,"\'-?:!;]', '', text)
    
    text = re.sub('[a-zA-Z]','', text)
    
    text = text.replace("\n", "")
    text = text.replace(" ", "")
    return text

def tokenization(text): # 斷詞
    seg_list = jieba.cut(text, cut_all=False)
    return list(seg_list)

In [70]:
news_extract = news_data.copy()
news_extract['prepreocessed'] = news_extract['Post_Content'].map(preprocess)
news_extract['tokens'] = news_extract['prepreocessed'].map(tokenization)
news_extract = news_extract.drop(columns=['Post_Content','prepreocessed'])
news_extract

,Publication_Date,tokens
0,2022-11-03,"[鴻海, 今日, 晚間, 宣布, 攜手, 沙烏地, 阿拉伯, 公共, 投資, 基金, 合資,..."
1,2022-11-03,"[財訊, 快報, 記者, 劉居全, 報導針, 對, 市場, 傳出, 中國, 已, 下令, 對..."
2,2022-11-02,"[河南, 鄭州, 疫情, 在, 月, 中旬, 擴散鴻海, 集團, 旗下, 富士康, 鄭州廠,..."
3,2022-11-03,"[鴻海, 旗下, 富智康, 日, 自結, 第季, 稅後虧, 損萬, 美元, 較, 去年同期,..."
4,2022-11-02,"[關注台, 股盤, 前, 要, 聞, 重點, 中國, 鄭州, 航空港, 經濟, 綜合, 實驗..."
...,...,...
17533,2022-12-15,"[產險, 防疫, 險理, 賠, 失血, 金額, 近尾聲, 資產重, 分類, 也將, 挹注, ..."
17534,2022-12-13,"[▲, 中環, 子公司, 賣陽明, 股票, 投資, 報酬, 率, %, 法人, 評落後大盤,..."
17535,2022-12-14,"[面對, 通膨時代, 退休金, 準備, 多少, 才夠, 中經院, 調查, 未, 退休, 族群..."
17536,2022-12-12,"[定存, 是, 最, 簡單, 的, 理財, 方式, 不, 需面, 臨虧, 損風險, 但, 相..."


In [69]:
fb_extract = fb_data.copy()
fb_extract['prepreocessed'] = fb_extract['Post_Content'].map(preprocess)
fb_extract['tokens'] = fb_extract['prepreocessed'].map(tokenization)
fb_extract = fb_extract.drop(columns=['Post_Content','prepreocessed'])
fb_extract

,Publication_Date,tokens
0,2022-12-02,"[相隔, 年, 摩洛哥, 再次, 闖進, 世界, 盃, 強, 🎉, 年卡達, 世界, 盃, ..."
1,2022-12-02,"[盧卡庫, 錯, 失太多, 機會, 比利, 時黃金, 一代, 可惜, 了, 😔, 年卡達, ..."
2,2022-12-02,"[疫情, 趨緩, 口罩, 令, 也, 逐漸, 解封, 國旅, 熱度, 跟, 著加, 溫台, ..."
3,2022-12-02,"[彰化, 福興鄉, 的, 同安, 村, 近期, 空氣, 中, 都, 散, 發惡, 臭, 也,..."
4,2022-12-01,"[歲, 公民, 權的, 修憲, 公投, 因為, 同意, 票遠, 低, 於, 門檻, 而, 宣..."
...,...,...
5632,2022-12-15,"[論壇, 商業, 周刊, 巨大, 今年, 前季, 淨利, 突破, 億元, 年增率, 達, 創..."
5633,2022-12-15,"[地震, 好搖, ⋯, 你, 有, 收到, 國家級, 警報, 嗎]"
5634,2022-12-15,"[美媒, 表示, 此舉, 是, 為, 了, 阻止, 它們, 購買, 某些, 美國, 零組件]"
5635,2022-12-15,"[范瑋琪, 即便, 萬般, 不, 捨, 您, 已, 卸下, 勞苦, 重擔, 安息, 主懷]"


# 3. Group to same date

In [80]:
news_condense = news_extract.groupby("Publication_Date").agg({'tokens':'sum'})
fb_condense = fb_extract.groupby("Publication_Date").agg({'tokens':'sum'})

In [83]:
news_condense = news_condense.reset_index()
fb_condense = fb_condense.reset_index()

In [126]:
tmp1 = news_condense.copy()
tmp2 = fb_condense.copy()
tmp1 = tmp1.merge(tmp2, on='Publication_Date', how='left')
combine_df = tmp1.copy()

sz = tmp1.shape[0]
combine_df['Tokens'] = ''
for i in range(sz):
    try:
        combine_df['Tokens'][i] = combine_df['tokens_x'][i] + combine_df['tokens_y'][i]
    except: 
        combine_df['Tokens'][i] = combine_df['tokens_x'][i]
combine_df = combine_df.drop(columns=['tokens_x', 'tokens_y'])
combine_df['token_num'] = combine_df['Tokens'].map(len)
combine_df

,Publication_Date,Tokens,token_num
0,2022-10-29,"[護國, 神山, 台積電, 近日, 內憂, 外患, 不斷, 市值, 也, 跌, 到, 兆, ...",54925
1,2022-10-30,"[本周, 迎來, 月, 的, 第一周, 鴻海, 預計將, 公布, 月, 營收, 由, 於, ...",104587
2,2022-10-31,"[聯發, 科上, 週五法, 說會釋, 出, 今年, 展望, 恐低, 於, 預期, 所幸, 已...",267911
3,2022-11-01,"[外資, 今日, 賣, 超台, 股, 億元, 由, 買, 轉賣, 台, 指期, 現貨淨, 多...",192944
4,2022-11-02,"[河南, 鄭州, 疫情, 在, 月, 中旬, 擴散鴻海, 集團, 旗下, 富士康, 鄭州廠,...",261753
5,2022-11-03,"[鴻海, 今日, 晚間, 宣布, 攜手, 沙烏地, 阿拉伯, 公共, 投資, 基金, 合資,...",396649
6,2022-11-04,"[鴻海進, 軍電動車, 再, 傳, 好消息, 日晚, 間鴻海, 發布, 與, 沙烏地, 阿拉...",81131
7,2022-11-05,"[我, 曾經, 在, 此, 專欄, 提過, 資金, 控管, 永遠, 是, 投資, 最, 重要...",29677
8,2022-11-06,"[台股, 進入, 財報, 法, 說, 會, 旺季, 本周, 聯詠鴻海國, 巨儒鴻等, 電子,...",58547
9,2022-11-07,"[蘋果, 公司, 與鴻海, 今日, 早上, 同步, 發出, 聲明, 顯示, 在, 紅色供, ...",142139


In [127]:
combine_df.describe()

,token_num
count,48.000000
mean,154595.833333
std,105998.862140
min,26183.000000
25%,73077.250000
50%,141860.000000
75%,230590.000000
max,396649.000000


# Sentiment Analysis

In [134]:
path = 'C:/Users/USER/Desktop/Prof_Hsieh_Project/about_emotion/Chinese_derivative/words_and_labels'
os.chdir(path)
Sentiment_Score = pd.read_pickle('Sentiment_Score.pkl')
labels = list(Sentiment_Score['token'])

In [136]:
Sentiment_Score

,token,Normalized_Sentiment_Score
0,筆記,0.537418
1,機械手,0.685253
2,吸,-2.186708
3,車庫,0.525293
4,艾滋病,0.370012
...,...,...
15235,堅定,-0.257872
15236,飢餓的,-0.257872
15237,盛行,-0.257872
15238,進攻,-0.257872


In [141]:
def Average(lst):
    return sum(lst) / len(lst)

def Count_Sentiment_Score(input_tokens, show=False):
    score = 0
    count = 0
    for token in input_tokens:
        if token in labels:
            same_token_list = list(Sentiment_Score[Sentiment_Score['token'] == token]['Normalized_Sentiment_Score'])
            if show:
                print(token, same_token_list)
            avg_score = Average(same_token_list)
            count += 1
            score += avg_score
    if(count == 0): return 0
    else: return score/count

In [142]:
Count_Sentiment_Score(combine_df['Tokens'][0])

不斷 [0.5993162850282143]
市值 [0.6852527853275587]
也 [0.6583810150207764, 0.5428157074630069, -0.5854477850290029]
到 [-0.018352209703342236]
兆 [-0.737658903652223]
根據 [-0.588205670277333, 0.45443828001162634]
最新 [0.3644400676117113]
專利 [0.1858841190950849, 0.2716149831640487]
申請 [-0.8228147949414311]
是 [-0.13151230160010807, -0.06865514124578379, -0.3571321829608377, 0.6131720624669326, -1.8662044617909432]
年 [-2.441019092777612, -0.35029856309195045, -0.9282709442440773, 0.4084493921600135, -0.7859592079240895]
來 [-0.41714609517399776, 0.43162870335927134]
旺盛 [0.5993162850282143, 0.5993162850282143]
的 [0.5872694094055224, -0.5931342539320813, 0.5405514036581035, 0.5531507738294686, -2.211510393988561]
出現 [-0.2769322434969442, 0.6852527853275587, 0.4339662178728399, 0.5993162850282143]
的 [0.5872694094055224, -0.5931342539320813, 0.5405514036581035, 0.5531507738294686, -2.211510393988561]
專利 [0.1858841190950849, 0.2716149831640487]
排名 [0.6177654352820723, 0.6852527853275587]
上 [0.498159094

特別 [0.21437080046673873, 0.5805067461186354, 0.5993162850282143]
邀請 [0.3464728119648287, 0.4730640537178517]
演講 [-1.443715909045535, -0.25787240413150314]
希望 [0.62647581712401, 0.6852527853275587, -0.04801836832468653, -0.9320259649429121, 0.6226090042655147, 0.5805067461186354, -1.4645749372772863, -4.611477423235554, 0.3983963914402621, 0.5778100928215403, 0.5993162850282143, 0.5993162850282143, 0.5993162850282143, 0.5993162850282143, 0.4755108155010181]
豐富 [0.5993162850282143, 0.5993162850282143, 0.5993162850282143, 0.5993162850282143, 0.5993162850282143, 0.5993162850282143, 0.5993162850282143, 0.5993162850282143, 0.5993162850282143]
的 [0.5872694094055224, -0.5931342539320813, 0.5405514036581035, 0.5531507738294686, -2.211510393988561]
經歷 [0.5727409215387148]
深刻 [0.5993162850282143]
的 [0.5872694094055224, -0.5931342539320813, 0.5405514036581035, 0.5531507738294686, -2.211510393988561]
經驗 [-0.8356422795742592]
讓 [0.573998549212554, 0.04846436002316605]
共同 [0.4755108155010181]
安全 [0.3

但 [-0.5920615553570631, 0.5163883145504379]
內 [-0.07946099027978053, -0.5854477850290029]
尺寸 [-0.04727147239916922]
此外 [0.5993162850282143]
公司 [-0.5139600357596434, -1.3969979032981164, -0.020958588700246267, -1.1657960058992685, -0.8250623385463666, -0.8609933360544482, 0.5993162850282143]
股東 [-1.7584844444601686, -4.439420069276933, -3.239054906541982, -1.444248709662067]
與 [0.6232762050271392, 0.467994882435026]
讓 [0.573998549212554, 0.04846436002316605]
安心 [0.43162870335927134]
工作 [-0.5742523730260511, -0.43677412715453995, 0.6852527853275587]
宣布 [0.4626771348611972, 0.04900263884099583, 0.03235357352667615, -0.11339645459696712, 0.6031670980072438, 0.5993162850282143, -0.25787240413150314]
自願 [0.4755108155010181]
月 [-0.31028176905542393, -0.5854477850290029]
日 [0.33379925408610733]
人 [-0.6849116246204502]
的 [0.5872694094055224, -0.5931342539320813, 0.5405514036581035, 0.5531507738294686, -2.211510393988561]
克服 [0.5684093483824691, -0.25787240413150314]
挑戰 [0.6135127283562114, 0.21

成長 [0.3104660162379772, 0.582862394986513]
年 [-2.441019092777612, -0.35029856309195045, -0.9282709442440773, 0.4084493921600135, -0.7859592079240895]
月 [-0.31028176905542393, -0.5854477850290029]
信號 [-1.1645955817934364, 0.3464728119648287, -1.4042795258956526, 0.5993162850282143]
分數 [0.17441545800620675, 0.5450101892209042, 0.5840009142706201]
日 [0.33379925408610733]
月 [-0.31028176905542393, -0.5854477850290029]
結果 [0.6852527853275587, 0.2484894360427924, 0.6031670980072438, -0.776804747028232, 0.3180504167702829, 0.3700115065129898]
判斷 [0.4512188510685174, 0.5333045383888321, 0.5993162850282143, 0.5993162850282143, 0.5993162850282143, 0.5993162850282143]
分數 [0.17441545800620675, 0.5450101892209042, 0.5840009142706201]
一月 [0.5238731491924608]
大 [-0.1542650551077935]
年 [-2.441019092777612, -0.35029856309195045, -0.9282709442440773, 0.4084493921600135, -0.7859592079240895]
月 [-0.31028176905542393, -0.5854477850290029]
低點 [-2.044418589573005, -0.5854477850290029]
表示 [0.49389582332832965,

在 [-0.14636750165968315, 0.5872694094055224, -1.918809312324529, 0.467994882435026, -0.7395336295448479, 0.6023519487679697]
智慧 [0.3305751233631073, 0.5993162850282143, 0.5993162850282143]
通訊 [0.6852527853275587, 0.5471380317544372]
設備 [0.5603976390987401, 0.5862036210652128, 0.5348114520562999]
下 [-0.5854477850290029, -0.5854477850290029]
但 [-0.5920615553570631, 0.5163883145504379]
挑戰 [0.6135127283562114, 0.21437080046673873, -0.5854477850290029, -0.25787240413150314]
預期 [0.5090302287562419, -0.5854477850290029]
被動 [-0.25787240413150314]
月 [-0.31028176905542393, -0.5854477850290029]
日 [0.33379925408610733]
說明 [-1.568571055274354]
在 [-0.14636750165968315, 0.5872694094055224, -1.918809312324529, 0.467994882435026, -0.7395336295448479, 0.6023519487679697]
市場 [-1.0741861457616475, 0.4385140456482971, -1.5624697126565827]
的 [0.5872694094055224, -0.5931342539320813, 0.5405514036581035, 0.5531507738294686, -2.211510393988561]
策略 [0.5294016689748163, 0.486943214300593, -0.25787240413150314]
日

...

人 [-0.6849116246204502]
遭受 [-1.0577337279530017, 0.5405514036581035, -0.5854477850290029]
到 [-0.018352209703342236]
嚴重 [-2.2294776497406787, 0.4952281756168193, -0.5854477850290029, -0.5854477850290029, -0.5854477850290029, -0.5854477850290029]
的 [0.5872694094055224, -0.5931342539320813, 0.5405514036581035, 0.5531507738294686, -2.211510393988561]
迫害 [-0.5854477850290029, -0.5854477850290029]
包括 [0.4084493921600135, 0.5252928291051029, 0.5382138891445347, 0.5568764576275017]
夫妻 [0.22986747498871807]
作家 [0.5932115515388556, 0.6852527853275587]
著名 [0.5993162850282143, 0.4755108155010181, 0.4755108155010181]
與 [0.6232762050271392, 0.467994882435026]
母親 [-0.5170921391832934, 0.5820890447483174]
一起 [0.49768449049823643]
自殺 [0.28692732168981605, -0.5854477850290029, -0.5854477850290029]
如果 [0.36685184791676234, 0.6852527853275587, 0.6852527853275587, 0.6852527853275587]
上海 [0.4626771348611972]
也 [0.6583810150207764, 0.5428157074630069, -0.5854477850290029]
是 [-0.13151230160010807, -0.06865514

過程 [0.2709321788367316]
向上 [0.6102639440962332, 0.5797787598112526, 0.2716149831640487, 0.4755108155010181]
最終 [-1.0072154314452624, 0.3951897848157108, 0.5993162850282143]
月 [-0.31028176905542393, -0.5854477850290029]
後 [-1.4406457049714072]
受到 [-0.25787240413150314]
拉 [-1.1318077059895875, -1.0299669154018698, -0.8679695102821898, -1.135984367184731]
但 [-0.5920615553570631, 0.5163883145504379]
指數 [-3.761860122551472, -0.8173503065630435]
創 [0.6852527853275587]
向上 [0.6102639440962332, 0.5797787598112526, 0.2716149831640487, 0.4755108155010181]
的 [0.5872694094055224, -0.5931342539320813, 0.5405514036581035, 0.5531507738294686, -2.211510393988561]
長 [0.5637307147521265, -0.25787240413150314]
趨勢 [-1.5644454674118946, -0.7937725948923468, 0.17472345579115525, 0.5405514036581035, 0.6852527853275587, 0.4512188510685174, 0.5993162850282143]
鎖定 [0.5959202327379727, 0.20942281364324603]
大 [-0.1542650551077935]
重點 [0.2837027490864301]
在 [-0.14636750165968315, 0.5872694094055224, -1.918809312324

的 [0.5872694094055224, -0.5931342539320813, 0.5405514036581035, 0.5531507738294686, -2.211510393988561]
表示 [0.49389582332832965, 0.5144869800711063, 0.4755108155010181]
在 [-0.14636750165968315, 0.5872694094055224, -1.918809312324529, 0.467994882435026, -0.7395336295448479, 0.6023519487679697]
美國 [-0.7738701865520763]
打 [0.1301060597384908, -0.33217320076980605, 0.5767626865579567, 0.5637307147521265]
的 [0.5872694094055224, -0.5931342539320813, 0.5405514036581035, 0.5531507738294686, -2.211510393988561]
並 [0.4385140456482971]
是 [-0.13151230160010807, -0.06865514124578379, -0.3571321829608377, 0.6131720624669326, -1.8662044617909432]
的 [0.5872694094055224, -0.5931342539320813, 0.5405514036581035, 0.5531507738294686, -2.211510393988561]
多 [-0.1736674326977901, 0.3755598339861672]
時間 [0.6852527853275587, 0.6852527853275587, 0.20177143029537342]
是 [-0.13151230160010807, -0.06865514124578379, -0.3571321829608377, 0.6131720624669326, -1.8662044617909432]
在 [-0.14636750165968315, 0.58726940940

加速 [0.5684093483824691, -1.65790360786394, 0.5471380317544372, 0.5993162850282143]
向 [-0.35534288952116483]
保險 [-0.3147606313941017]
公司 [-0.5139600357596434, -1.3969979032981164, -0.020958588700246267, -1.1657960058992685, -0.8250623385463666, -0.8609933360544482, 0.5993162850282143]
正式 [-1.1817038754292912]
月 [-0.31028176905542393, -0.5854477850290029]
開始 [0.41663386152084714, 0.03788441899527341, -0.34642791737624706, 0.17808359843347793, -1.724649098304821, -0.5323757193884833, 0.12740154583236812, -0.4300481421496129, -0.6670315000193533, 0.5333045383888321]
申請 [-0.8228147949414311]
執照 [0.5420688604716117]
反 [-0.5854477850290029]
進入 [-1.6944888814217531, -0.24286752090595232, 0.5357995519526686, -0.8467556125323031]
波動 [0.6073785164254178, 0.6852527853275587, 0.6852527853275587]
出現 [-0.2769322434969442, 0.6852527853275587, 0.4339662178728399, 0.5993162850282143]
明確 [0.5993162850282143, 0.5993162850282143, 0.5993162850282143, 0.4755108155010181, 0.4755108155010181]
反向 [-0.5854477850

-0.002497905701218398

In [143]:
combine_df['Sentiment_Score'] = combine_df['Tokens'].map(Count_Sentiment_Score)

In [144]:
combine_df

,Publication_Date,Tokens,token_num,Sentiment_Score
0,2022-10-29,"[護國, 神山, 台積電, 近日, 內憂, 外患, 不斷, 市值, 也, 跌, 到, 兆, ...",54925,-0.002498
1,2022-10-30,"[本周, 迎來, 月, 的, 第一周, 鴻海, 預計將, 公布, 月, 營收, 由, 於, ...",104587,-0.055003
2,2022-10-31,"[聯發, 科上, 週五法, 說會釋, 出, 今年, 展望, 恐低, 於, 預期, 所幸, 已...",267911,-0.039743
3,2022-11-01,"[外資, 今日, 賣, 超台, 股, 億元, 由, 買, 轉賣, 台, 指期, 現貨淨, 多...",192944,-0.023583
4,2022-11-02,"[河南, 鄭州, 疫情, 在, 月, 中旬, 擴散鴻海, 集團, 旗下, 富士康, 鄭州廠,...",261753,-0.038870
5,2022-11-03,"[鴻海, 今日, 晚間, 宣布, 攜手, 沙烏地, 阿拉伯, 公共, 投資, 基金, 合資,...",396649,-0.040775
6,2022-11-04,"[鴻海進, 軍電動車, 再, 傳, 好消息, 日晚, 間鴻海, 發布, 與, 沙烏地, 阿拉...",81131,-0.031491
7,2022-11-05,"[我, 曾經, 在, 此, 專欄, 提過, 資金, 控管, 永遠, 是, 投資, 最, 重要...",29677,-0.026696
8,2022-11-06,"[台股, 進入, 財報, 法, 說, 會, 旺季, 本周, 聯詠鴻海國, 巨儒鴻等, 電子,...",58547,-0.017016
9,2022-11-07,"[蘋果, 公司, 與鴻海, 今日, 早上, 同步, 發出, 聲明, 顯示, 在, 紅色供, ...",142139,-0.047300


In [145]:
# combine_df.to_pickle('Sentiment_Analysis_Result.pkl')